In [1]:
from text_analysis_funcs import *
import datetime , os
import pyfpgrowth ,jieba
from math import ceil
jieba.suggest_freq('韓國瑜', True)
jieba.suggest_freq('兩岸一家親' , True)

PUNC = "[0-9a-zA-Z\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）+「」，。）（〔〕／『』:\[\]’：\-《》％‘○]"


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Nacu\AppData\Local\Temp\jieba.cache
Loading model cost 0.387 seconds.
Prefix dict has been built succesfully.


In [24]:
def get_dict(keyword,source):
    
    my_dict = shared_dict()
    
    dir_name = keyword + '_' + source
    
    file_names = os.listdir(dir_name)
    
    for file_name in file_names:
        f = open( dir_name +'/'+file_name , 'r' , encoding='utf-8')
        lines = f.readlines()
        for text in lines:
            text = re.sub(PUNC , "" , text)
            text_list = list( jieba.cut(text) )
            my_dict.add_doc(text_list)
            
    return my_dict

def get_transcation_by_time(DICT , keyword , source , s_time , e_time):
    
    cur_time = s_time
    
    dir_name = keyword + '_' + source
    text_lists = []
    while cur_time <= e_time:
        Year = str(cur_time.year)
        Month = str(cur_time.month)
        Day = str(cur_time.day)
        
        if(source != '自由' ):
            Month = '0'+Month
            if(len(Day) < 2):
                Day = '0'+Day
        
        file_name = dir_name+ '/' + keyword + '-' + Year + '-' + Month + '-' + Day+'.txt'
        #print(file_name)
        try:
            f = open(file_name ,'r' , encoding='utf-8')
        except FileNotFoundError :
            cur_time = cur_time + datetime.timedelta(days = 1)
            continue
        lines = f.readlines()
        for text in lines:
            text = re.sub(PUNC , "" , text)
            text_list = list( jieba.cut(text) )
            text_lists.append(text_list)
        cur_time = cur_time + datetime.timedelta(days = 1)
        
    transcation = DICT.get_transcation_db(text_lists) 
    return transcation , len(transcation)

def get_pattern( transcation , min_support ):

    thold = max(5 , ceil(min_support * len(transcation)))
    thold = min(thold , len(transcation)*.5)
    print('min support',thold)
    pattern = pyfpgrowth.find_frequent_patterns(transcation,thold)
    return pattern

In [17]:
LTN = '自由'
CT = '中時'
keyword = '張安樂'

In [18]:
ltn_dict = get_dict(keyword , LTN)
ct_dict = get_dict(keyword , CT)

In [19]:
s_time = datetime.datetime(2018,6,)
e_time = datetime.datetime(2018,6,16)

ct_trans , ct_num = get_transcation_by_time(ct_dict,keyword,CT , s_time , e_time)
ltn_trans , ltn_num = get_transcation_by_time(ltn_dict , keyword , LTN , s_time , e_time)
ct_num,ltn_num

(4, 13)

In [25]:
ct_pattern = get_pattern(ct_trans , 0.01)
ltn_pattern = get_pattern(ltn_trans , 0.01)

min support 2.0
min support 5


32

205

In [26]:
print_frp(ct_dict , ct_pattern , ct_num)

畢業典禮 本校 0.5
市長獎 本校 0.5
頒發 本校 0.5
本校 家長 0.5
市長獎 畢業典禮 本校 0.5
畢業典禮 頒發 本校 0.5
畢業典禮 本校 家長 0.5
市長獎 頒發 本校 0.5
市長獎 本校 家長 0.5
頒發 本校 家長 0.5
市長獎 畢業典禮 頒發 本校 0.5
市長獎 畢業典禮 本校 家長 0.5
畢業典禮 頒發 本校 家長 0.5
市長獎 頒發 本校 家長 0.5
市長獎 畢業典禮 頒發 本校 家長 0.5
實小 畢業典禮 0.5
實小 市長獎 0.5
實小 頒發 0.5
實小 家長 0.5
實小 市長獎 畢業典禮 0.5
實小 畢業典禮 頒發 0.5
實小 畢業典禮 家長 0.5
實小 市長獎 頒發 0.5
實小 市長獎 家長 0.5
實小 頒發 家長 0.5
實小 市長獎 畢業典禮 頒發 0.5
實小 市長獎 畢業典禮 家長 0.5
實小 畢業典禮 頒發 家長 0.5
實小 市長獎 頒發 家長 0.5
實小 市長獎 畢業典禮 頒發 家長 0.5
國北教大 畢業典禮 0.5
國北教大 市長獎 0.5
國北教大 頒發 0.5
國北教大 家長 0.5
國北教大 市長獎 畢業典禮 0.5
國北教大 畢業典禮 頒發 0.5
國北教大 畢業典禮 家長 0.5
國北教大 市長獎 頒發 0.5
國北教大 市長獎 家長 0.5
國北教大 頒發 家長 0.5
國北教大 市長獎 畢業典禮 頒發 0.5
國北教大 市長獎 畢業典禮 家長 0.5
國北教大 畢業典禮 頒發 家長 0.5
國北教大 市長獎 頒發 家長 0.5
國北教大 市長獎 畢業典禮 頒發 家長 0.5
致歉 國北教大 0.5
致歉 畢業典禮 0.5
致歉 市長獎 0.5
致歉 頒發 0.5
致歉 家長 0.5
致歉 國北教大 畢業典禮 0.5
致歉 國北教大 市長獎 0.5
致歉 國北教大 頒發 0.5
致歉 國北教大 家長 0.5
致歉 市長獎 畢業典禮 0.5
致歉 畢業典禮 頒發 0.5
致歉 畢業典禮 家長 0.5
致歉 市長獎 頒發 0.5
致歉 市長獎 家長 0.5
致歉 頒發 家長 0.5
致歉 國北教大 市長獎 畢業典禮 0.5
致歉 國北教大 畢業典禮 頒發 0.5
致歉 國北教大 畢業典禮 家長 0.5
致歉 國北教大 市長獎 頒發 0.5
致歉 國北教大 市長

In [27]:
print_frp(ltn_dict , ltn_pattern , ltn_num)

教育 市長獎 0.38461538461538464
典禮 頒獎 0.38461538461538464
頒獎 國北教大 0.38461538461538464
學校 頒發 0.46153846153846156
學校 頒獎 0.46153846153846156
學校 頒發 頒獎 0.46153846153846156
上台 家長 0.38461538461538464
上台 家長 頒獎 0.38461538461538464
上台 頒發 0.38461538461538464
上台 頒發 頒獎 0.38461538461538464
上台 頒獎 0.46153846153846156
頒發 家長 0.38461538461538464
頒發 市長獎 家長 0.38461538461538464
市長獎 家長 0.46153846153846156
家長 頒獎 0.46153846153846156
市長獎 家長 頒獎 0.38461538461538464
畢業典禮 頒獎 0.46153846153846156
頒發 市長獎 0.46153846153846156
頒發 市長獎 頒獎 0.38461538461538464
頒發 頒獎 0.5384615384615384
市長獎 頒獎 0.5384615384615384
